In [431]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_colwidth',1000)
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
import seaborn as sns
import itertools
from __future__ import division
from numpy import product
from sklearn.model_selection import StratifiedKFold
from xgboost.sklearn import XGBClassifier

In [103]:
import nltk
import spacy
nlp=spacy.load('en')

** Reading the training data set **

In [104]:
df=pd.read_csv("train.csv",encoding='utf-8')

In [105]:
df.head(10)

,id,text,author
0,id26305,"This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.",EAP
1,id17569,It never once occurred to me that the fumbling might be a mere mistake.,HPL
2,id11008,"In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.",EAP
3,id27763,"How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.",MWS
4,id12958,"Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk.",HPL
5,id22965,"A youth passed in solitude, my best years spent under your gentle and feminine fosterage, has so refined the groundwork of my character that I cannot overcome an intense distaste to the usual brutality exercised on board ship: I have never believed it to be necessary, and when I heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew, I felt myself peculiarly fortunate in being able to secure his services.",MWS
6,id09674,"The astronomer, perhaps, at this point, took refuge in the suggestion of non luminosity; and here analogy was suddenly let fall.",EAP
7,id13515,The surcingle hung in ribands from my body.,EAP
8,id19322,"I knew that you could not say to yourself 'stereotomy' without being brought to think of atomies, and thus of the theories of Epicurus; and since, when we discussed this subject not very long ago, I mentioned to you how singularly, yet with how little notice, the vague guesses of that noble Greek had met with confirmation in the late nebular cosmogony, I felt that you could not avoid casting your eyes upward to the great nebula in Orion, and I certainly expected that you would do so.",EAP
9,id00912,"I confess that neither the structure of languages, nor the code of governments, nor the politics of various states possessed attractions for me.",MWS


In [106]:
len(df['text'])

19579

In [107]:
df_sub=pd.read_csv("test.csv",encoding='utf-8')

In [108]:
df_total=pd.concat([df,df_sub])

In [109]:
df_total.head(10)

,author,id,text
0,EAP,id26305,"This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall."
1,HPL,id17569,It never once occurred to me that the fumbling might be a mere mistake.
2,EAP,id11008,"In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction."
3,MWS,id27763,"How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair."
4,HPL,id12958,"Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk."
5,MWS,id22965,"A youth passed in solitude, my best years spent under your gentle and feminine fosterage, has so refined the groundwork of my character that I cannot overcome an intense distaste to the usual brutality exercised on board ship: I have never believed it to be necessary, and when I heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew, I felt myself peculiarly fortunate in being able to secure his services."
6,EAP,id09674,"The astronomer, perhaps, at this point, took refuge in the suggestion of non luminosity; and here analogy was suddenly let fall."
7,EAP,id13515,The surcingle hung in ribands from my body.
8,EAP,id19322,"I knew that you could not say to yourself 'stereotomy' without being brought to think of atomies, and thus of the theories of Epicurus; and since, when we discussed this subject not very long ago, I mentioned to you how singularly, yet with how little notice, the vague guesses of that noble Greek had met with confirmation in the late nebular cosmogony, I felt that you could not avoid casting your eyes upward to the great nebula in Orion, and I certainly expected that you would do so."
9,MWS,id00912,"I confess that neither the structure of languages, nor the code of governments, nor the politics of various states possessed attractions for me."


**Modelling Approach:**
    
  1) Tfidf vectorizer and Count verctorizer will extract various word based features within the text, we can predict the author  based  on these word based features.
  
   2)Meta features such as sentence length , punctutaion frequency , max length of words in sentence will be used to build a predictive model.
   
   3) Word pattern based on Markov chains will be used to develop a predictve model.
   
   4) Finally a stacked model from Tfidf Vectors , Countvectorizer,Markov Chain ,Meta features and LSI features will be trained on XG boost to  
    

** TfIdf Vectorrizer **

tfIdf vectorizer converts the text into word based features. It will develop a matrix of shape(n_docs,vocab size). The matrix will be filled with word frequency * log(Total Number documents/Number of documents in which this word is present). By normalizing with the invrese log of the document frequecy, the word frequency will be normalised take care of features that are present in all documents and are less predictive.

In [110]:
Vectorizer=TfidfVectorizer(stop_words='english')

In [111]:
X_total=Vectorizer.fit_transform(df_total['text'].get_values())

In [112]:
X=Vectorizer.transform(df['text'].get_values())
X_sub=Vectorizer.transform(df_sub['text'].get_values())

In [113]:
X_total

<27971x27996 sparse matrix of type '<type 'numpy.float64'>'
	with 313932 stored elements in Compressed Sparse Row format>

Test and train Data seperation 

In [114]:
df['Target']=df['author'].replace(['EAP','HPL','MWS'],[0,1,2])

In [115]:
Y=df['Target'].get_values()

In [116]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,stratify=Y,random_state=42)

In [117]:
mnb=MultinomialNB()
parameters={'alpha':[0.0001,0.001,0.01,0.1,0.5,0.75,1,5,10]}
clf=GridSearchCV(mnb,cv=5,param_grid=parameters,scoring='log_loss')
clf.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='log_loss', verbose=0)

In [131]:
clf.best_params_

{'alpha': 0.01}

In [136]:
Y_predicted=clf.predict(X_test)

In [137]:
accuracy_score(Y_test,Y_predicted)

0.8126659856996936

In [138]:
log_loss(Y_test,clf.predict_proba(X_test))

0.4604502568209501

** Calibration of TfIdf Vectorizer model **

Probabilities given by Naive bayes algorithm are not accurate.So, calibrating the probabilities imporve the confidence of probabilities thrown by out by the classifier

In [123]:
X_test_ca,X_test_va,Y_test_ca,Y_test_va=train_test_split(X_test,Y_test,test_size=0.5,stratify=Y_test,random_state=42)

In [124]:
calib_clf=CalibratedClassifierCV(clf,method='sigmoid',cv='prefit')
calib_clf.fit(X_test_ca,Y_test_ca)
calibrated_proba=calib_clf.predict_proba(X_test_va)

In [125]:
log_loss(Y_test_va,calibrated_proba)

0.49930310623484386

In [126]:
accuracy_score(Y_test_ca,calib_clf.predict(X_test_ca))

0.8222313036371066

The accuracy of both the calibrated and base model is almost the same.However, the logloss score has dropped for the calibrated 
classifier.

** Count Vectorizer Model **

In [127]:
cvt=CountVectorizer()
cvt.fit(df_total['text'].get_values())
Xcvt=cvt.transform(df['text'].get_values())
Ycvt=df['Target'].get_values()
Xcvt_train,Xcvt_test,Ycvt_train,Ycvt_test=train_test_split(Xcvt,Ycvt,test_size=0.25,stratify=Y,random_state=42)
parameters_cvt={'alpha':[0.001,0.01,0.1,1,10,100]}
Mnb2=MultinomialNB()
clf_cvt=GridSearchCV(Mnb2,cv=5,param_grid=parameters_cvt,scoring='log_loss')
clf_cvt.fit(Xcvt_train,Ycvt_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='log_loss', verbose=0)

In [128]:
log_loss(Ycvt_test,clf_cvt.predict_proba(Xcvt_test))

0.4810625307001191

In [129]:
accuracy_score(Ycvt_test,clf_cvt.predict(Xcvt_test))

0.839223697650664

In [130]:
clf_cvt.best_params_

{'alpha': 1}

** Calibrating probabilities for Count vectorizer **

In [140]:
Xcvt_train_ca,Xcvt_test_ca,Ycvt_train_ca,Ycvt_test_ca=train_test_split(Xcvt,Ycvt,test_size=0.25,stratify=Ycvt,random_state=42)
clf_cvt_calib=CalibratedClassifierCV(clf_cvt,method='sigmoid',cv='prefit')
clf_cvt_calib.fit(Xcvt_train_ca,Ycvt_train_ca)
cvt_ca_proba=clf_cvt_calib.predict_proba(Xcvt_test_ca)
cvt_ca_predict=clf_cvt_calib.predict(Xcvt_test_ca)

In [141]:
log_loss(Ycvt_test_ca,cvt_ca_proba)

0.4833647175025266

In [142]:
accuracy_score(Ycvt_test_ca,cvt_ca_predict)

0.8406537282941777

Over allthe Calibrated Count vectorizer model has higher accuracy of 84 %.

** Markov Chain Models **

In Naive bayes algorithm, we assume that the features are independent. This assumption might not be correct for all data sets. On the other hand,in Markov models of order n, we assume that each feature is dependent on n preceeding features. In the current context, each author might have specific usage of words and punctutaions. This will be visible in the conditional frequency of a character based on the previous n characters. How ever, we are not sure which n will work for this specific problem. So, the best option is to try differnt (n) and shall select the one that gives best prediction accuracy.

In [143]:
df[['id','text','author','Target']].head(10)

,id,text,author,Target
0,id26305,"This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.",EAP,0
1,id17569,It never once occurred to me that the fumbling might be a mere mistake.,HPL,1
2,id11008,"In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.",EAP,0
3,id27763,"How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.",MWS,2
4,id12958,"Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk.",HPL,1
5,id22965,"A youth passed in solitude, my best years spent under your gentle and feminine fosterage, has so refined the groundwork of my character that I cannot overcome an intense distaste to the usual brutality exercised on board ship: I have never believed it to be necessary, and when I heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew, I felt myself peculiarly fortunate in being able to secure his services.",MWS,2
6,id09674,"The astronomer, perhaps, at this point, took refuge in the suggestion of non luminosity; and here analogy was suddenly let fall.",EAP,0
7,id13515,The surcingle hung in ribands from my body.,EAP,0
8,id19322,"I knew that you could not say to yourself 'stereotomy' without being brought to think of atomies, and thus of the theories of Epicurus; and since, when we discussed this subject not very long ago, I mentioned to you how singularly, yet with how little notice, the vague guesses of that noble Greek had met with confirmation in the late nebular cosmogony, I felt that you could not avoid casting your eyes upward to the great nebula in Orion, and I certainly expected that you would do so.",EAP,0
9,id00912,"I confess that neither the structure of languages, nor the code of governments, nor the politics of various states possessed attractions for me.",MWS,2


** Train and Test split for Markov models **

In [144]:
Xmk_train,Xmk_test,Ymk_train,Ymk_test=train_test_split(df['text'].get_values(),df['author'].get_values(),test_size=0.25,stratify=Ycvt)

** Function to create character n grams **

In [146]:
def Conditional_n_grams(text_columns_list,class_column_list,ngram):
    ###################################################################################
    def Create_Char_n_Grams(string,n):
        text=(string)
        gram_len=n
        list_ngrams=[]
        for p in np.arange(len(text)):
            if p+gram_len>len(text):
                break
            else:    
                list_ngrams.append(text[p:p+gram_len])
        return(list_ngrams)
    Vector_Char_n_Grams=np.vectorize(Create_Char_n_Grams)
    ##################################################################################################
    df=pd.DataFrame({'text':text_columns_list,'class':class_column_list})
    df['Char_n_Grams']=df['text'].apply(lambda x : Vector_Char_n_Grams(x,ngram))
    list_ngram=[]
    list_author=[]
    for k in df['class'].unique():
        list_Char_n_Grams=list(itertools.chain.from_iterable(list(df['Char_n_Grams'][df['class']==k])))
        list_Author=[k]*len(list_Char_n_Grams)
        list_ngram.append(list_Char_n_Grams)
        list_author.append(list_Author)
    list_ngram=list(itertools.chain.from_iterable(list_ngram))
    list_author=list(itertools.chain.from_iterable(list_author))
    df_n_grams=pd.DataFrame({'Ngram':list_ngram,'Author':list_author})
    df_n_grams['Precedent']=df_n_grams['Ngram'].apply(lambda x : x[:-1])
    df_n_grams['Antecedent']=df_n_grams['Ngram'].apply(lambda x : x[-1:])
    df_n_grams['Ngram_Length']=df_n_grams['Ngram'].apply(lambda x : len(x))
    df_n_grams['Precedent_Count']=df_n_grams.groupby(['Author','Precedent'])['Ngram'].transform(lambda x : len(x))
    df_n_grams['Cond_Antecedent_Count']=df_n_grams.groupby(['Author','Precedent','Antecedent'])['Ngram'].transform(lambda x : len(x))
    df_n_grams['Conditional_Probability']=df_n_grams['Cond_Antecedent_Count']/df_n_grams['Precedent_Count']
    df_n_grams.drop_duplicates(keep='first',inplace=True)
    return(df_n_grams)

In [251]:
dataframe=Conditional_n_grams(list(Xmk_train),list(Ymk_train),4)

In [252]:
dataframe.head(10)

,Author,Ngram,Precedent,Antecedent,Ngram_Length,Precedent_Count,Cond_Antecedent_Count,Conditional_Probability
0,HPL,Upon,Upo,n,4,15,15,1.000000
1,HPL,pon,pon,,4,186,138,0.741935
2,HPL,on t,on,t,4,1839,511,0.277868
3,HPL,n th,n t,h,4,2066,1720,0.832527
4,HPL,the,th,e,4,12271,9060,0.738326
5,HPL,the,the,,4,9772,7719,0.789910
6,HPL,he d,he,d,4,9210,496,0.053855
7,HPL,e de,e d,e,4,833,231,0.277311
8,HPL,dea,de,a,4,999,154,0.154154
9,HPL,deat,dea,t,4,184,75,0.407609


** Predicting for a sentence **

In [310]:
def Target_Prediction(list_text,ngram,prior=dataframe,alpha=0.1):
    
    def Create_Char_n_Grams(string,n):
        text=(string)
        gram_len=n
        list_ngrams=[]
        for p in np.arange(len(text)):
            if p+gram_len>len(text):
                break
            else:    
                list_ngrams.append(text[p:p+gram_len])
        return(list_ngrams)
    Vector_Char_n_Grams=np.vectorize(Create_Char_n_Grams)
    #########################################################################################################################
    df_text=pd.DataFrame({'Text':list_text})
    df_text['list_ngram']=df_text['Text'].apply(lambda x : Vector_Char_n_Grams(x,ngram))
   ##########################################################################################################################
    def Changing_to_Data_Frame(list_text1,prior1=prior,alpha=0.1):
        df_EAP=prior1[prior1['Author']=='EAP']
        df_HPL=prior1[prior1['Author']=='HPL']
        df_MWS=prior1[prior1['Author']=='MWS']
        df_test=pd.DataFrame({'Ngram':list_text1})
        df_EAP_temp=pd.merge(df_test,df_EAP,left_on='Ngram',right_on='Ngram',how='left')
        df_EAP_temp=df_EAP_temp[['Author','Ngram','Conditional_Probability']]
        df_EAP_temp['Conditional_Probability'][df_EAP_temp['Conditional_Probability'].isnull()]=alpha
        df_HPL_temp=pd.merge(df_test,df_HPL,left_on='Ngram',right_on='Ngram',how='left')
        df_HPL_temp=df_HPL_temp[['Author','Ngram','Conditional_Probability']]
        df_HPL_temp['Conditional_Probability'][df_HPL_temp['Conditional_Probability'].isnull()]=alpha
        df_MWS_temp=pd.merge(df_test,df_MWS,left_on='Ngram',right_on='Ngram',how='left')
        df_MWS_temp=df_MWS_temp[['Author','Ngram','Conditional_Probability']]
        df_MWS_temp['Conditional_Probability'][df_MWS_temp['Conditional_Probability'].isnull()]=alpha
        EAP=sum(np.log(df_EAP_temp['Conditional_Probability']))+np.log(1/3)
        HPL=sum(np.log(df_HPL_temp['Conditional_Probability']))+np.log(1/3)
        MWS=sum(np.log(df_MWS_temp['Conditional_Probability']))+np.log(1/3)
        probability_EAP=1/(1+np.exp(HPL-EAP)+np.exp(MWS-EAP))
        probability_HPL=1/(1+np.exp(EAP-HPL)+np.exp(MWS-HPL))
        probability_MWS=1/(1+np.exp(EAP-MWS)+np.exp(HPL-MWS))
        return(probability_EAP,probability_HPL,probability_MWS)
    #Vectorize_Changing_to_Data_Frame=np.vectorize(Changing_to_Data_Frame)
  ###########################################################################################################################
    EAP_probability=[]
    HPL_probability=[]
    MWS_probability=[]
    count=0
    for k in df_text['list_ngram']:
        A1,A2,A3=Changing_to_Data_Frame(k)
        count+=1
        if count%1000==0:
            print(count)
        EAP_probability.append(A1)
        HPL_probability.append(A2)
        MWS_probability.append(A3)
    df_output=pd.DataFrame({'EAP':EAP_probability,'HPL':HPL_probability,'MWS':MWS_probability}) 
    #df_text['Probabilities']=df_text['list_ngram'].apply(lambda x : Vectorize_Changing_to_Data_Frame(x))
   ##########################################################################################################################
    #output_array=Vectorize_Changing_to_Data_Frame(df_text['list_ngram'].get_values())
    
    return(df_output.get_values())  

In [254]:
mk_prediction=Target_Prediction(Xmk_test,4,prior=dataframe,alpha=0.1)

1000
2000
3000
4000


In [255]:
mk_predi_labels=np.argmax(mk_prediction.get_values(),axis=1)

In [256]:
Ymk_test_labels=[]
for p in Ymk_test:
    if p=='EAP':
        Ymk_test_labels.append(0) 
    elif p=='HPL':
        Ymk_test_labels.append(1)
    else:
        Ymk_test_labels.append(2)
     

In [257]:
accuracy_score(Ymk_test_labels,mk_predi_labels)

0.789785495403473

In [258]:
log_loss(Ymk_test_labels,mk_prediction.get_values())

1.0075777542162356

Markov models are giving an accuracy of 79% which is close to the prediction accuracy of TfIdf and Count vectorizer based models.
But the log loss is very low compared to both of them. 

We can try different n gram size and alpha values and see if it imporves the prediction accuracy


**Stacked Models**

In [267]:
df.head(5)

,id,text,author,Target
0,id26305,"This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.",EAP,0
1,id17569,It never once occurred to me that the fumbling might be a mere mistake.,HPL,1
2,id11008,"In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.",EAP,0
3,id27763,"How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair.",MWS,2
4,id12958,"Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk.",HPL,1


In [313]:
X_stack=df['text'].get_values()
Y_stack=df['Target'].get_values()
Y_author_stack=df['author'].get_values()

In [315]:
Skf=StratifiedKFold(n_splits=5,shuffle=False,random_state=42)
X_tfidf_stack=np.zeros((Y_stack.shape[0],2))
X_cvt_stack=np.zeros((Y_stack.shape[0],2))
X_mk_stack=np.zeros((Y_stack.shape[0],2))
for train_index,test_index in Skf.split(X_stack,Y_stack):
    #Multinominal Naive Bayes Model based on TfIdf
    ######################################################################################################################
    Vectorizer=TfidfVectorizer(stop_words='english')
    X_train_tfidf=Vectorizer.fit_transform(X_stack[train_index])
    Y_train_tfidf=Y_stack[train_index]
    X_test_tfidf=Vectorizer.transform(X_stack[test_index])
    mnb_tfidf=MultinomialNB(alpha=0.01)
    mnb_tfidf.fit(X_train_tfidf,Y_train_tfidf)
    predict_probability=mnb_tfidf.predict_proba(X_test_tfidf)
    X_tfidf_stack[test_index,0]=predict_probability[:,0]
    X_tfidf_stack[test_index,1]=predict_probability[:,1]
   #Multinominal Naive Bayes based on CountVectorizer
    #####################################################################################################################
    cvt_Vectorizer=CountVectorizer()
    X_train_cvt=cvt_Vectorizer.fit_transform(X_stack[train_index])
    Y_train_cvt=Y_stack[train_index]
    X_test_cvt=cvt_Vectorizer.transform(X_stack[test_index])
    mnb_cvt=MultinomialNB(alpha=1)
    mnb_cvt.fit(X_train_cvt,Y_train_cvt)
    predict_proba_cvt=mnb_cvt.predict_proba(X_test_cvt)
    X_cvt_stack[test_index,0]=predict_proba_cvt[:,0]
    X_cvt_stack[test_index,1]=predict_proba_cvt[:,1]
   #####################################################################################################################
    dataframe=Conditional_n_grams(list(X_stack[train_index]),list(Y_author_stack[train_index]),4)
    predict_proba_mk=Target_Prediction(X_stack[test_index],4,prior=dataframe,alpha=0.1)
    X_mk_stack[test_index,0]=predict_proba_mk[:,0]
    X_mk_stack[test_index,1]=predict_proba_mk[:,1]

1000
2000
3000
1000
2000
3000
1000
2000
3000
1000
2000
3000
1000
2000
3000


In [316]:
df_TfidfNb=pd.DataFrame(X_tfidf_stack,columns=['Tfidf_feat1','Tfidf_feat2'])

In [318]:
df_TfidfNb['Tfidf_feat1']=df_TfidfNb['Tfidf_feat1'].apply(lambda x : np.round(x,decimals=2))
df_TfidfNb['Tfidf_feat2']=df_TfidfNb['Tfidf_feat2'].apply(lambda x : np.round(x,decimals=2))

In [319]:
df_TfidfNb.head(10)

,Tfidf_feat1,Tfidf_feat2
0,0.99,0.00
1,0.20,0.76
2,0.96,0.04
3,0.00,0.00
4,0.61,0.36
5,0.00,0.00
6,0.79,0.14
7,0.98,0.01
8,0.98,0.02
9,0.29,0.03


In [320]:
df_cvtNb=pd.DataFrame(X_cvt_stack,columns=['cvtNb_feat1','cvtNb_feat2'])

In [321]:
df_cvtNb['cvtNb_feat1']=df_cvtNb['cvtNb_feat1'].apply(lambda x :np.round(x,2))
df_cvtNb['cvtNb_feat2']=df_cvtNb['cvtNb_feat2'].apply(lambda x :np.round(x,2))

In [322]:
df_cvtNb.head(10)

,cvtNb_feat1,cvtNb_feat2
0,1.00,0.00
1,0.88,0.09
2,0.98,0.02
3,0.00,0.00
4,0.63,0.37
5,0.00,0.00
6,0.99,0.01
7,0.93,0.04
8,1.00,0.00
9,0.16,0.02


In [323]:
df_mk=pd.DataFrame(X_mk_stack,columns=['mk_feat1','mk_feat2'])

In [324]:
df_mk['mk_feat1']=df_mk['mk_feat1'].apply(lambda x :np.round(x,2))
df_mk['mk_feat2']=df_mk['mk_feat2'].apply(lambda x :np.round(x,2))

In [325]:
df_mk.head(10)

,mk_feat1,mk_feat2
0,1.00,0.00
1,0.00,1.00
2,1.00,0.00
3,0.00,0.00
4,0.53,0.46
5,0.00,0.00
6,1.00,0.00
7,0.81,0.10
8,1.00,0.00
9,0.00,0.00


In [326]:
df_stacked=pd.concat([df_TfidfNb,df_cvtNb,df_mk],axis=1)

In [327]:
df_stacked.head(10)

,Tfidf_feat1,Tfidf_feat2,cvtNb_feat1,cvtNb_feat2,mk_feat1,mk_feat2
0,0.99,0.00,1.00,0.00,1.00,0.00
1,0.20,0.76,0.88,0.09,0.00,1.00
2,0.96,0.04,0.98,0.02,1.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00
4,0.61,0.36,0.63,0.37,0.53,0.46
5,0.00,0.00,0.00,0.00,0.00,0.00
6,0.79,0.14,0.99,0.01,1.00,0.00
7,0.98,0.01,0.93,0.04,0.81,0.10
8,0.98,0.02,1.00,0.00,1.00,0.00
9,0.29,0.03,0.16,0.02,0.00,0.00


**Features from Latent Semantec Indexing **

Tfidf vectorizers generates sparse matrices.By using svd, we compress the over all dimensionality to 100 and in those dimensions, each document will be represented as a data point.  

In [352]:
Vectorizer_stack=TfidfVectorizer(stop_words='english')
X_stack_Tfidf=Vectorizer_stack.fit_transform(X_stack)

In [353]:
svd=TruncatedSVD(n_components=30,algorithm='arpack',random_state=42)
X_svd=svd.fit_transform(X_stack_Tfidf)

In [356]:
svd_columns=[]
for p in np.arange(30):
    svd_columns.append('svd_columns_'+str(p))

In [357]:
df_svd_features=pd.DataFrame(X_svd,columns=svd_columns)

** Meta features **

From each sentence we will extract various meta features such as sentence length, max size of words ,number of punctuations,
number of each punctuation,number of adjectives,number of words unique to each author. All these features will be added to other
features to build the predictive model.

In [358]:
df['n_words']=df['text'].apply(lambda x : len(x.split(" ")))

In [359]:
df['Period']=df['text'].str.count('\.')

In [360]:
df['Comma']=df['text'].str.count(',')

In [361]:
df['Question_Marks']=df['text'].str.count(r'\?')

In [362]:
df['Colon']=df['text'].str.count(r':')

In [363]:
df['Semicolon']=df['text'].str.count(';')

In [364]:
df['Quotation_Mark']=df['text'].str.count('"')

In [365]:
df['Aphostrophe']=df['text'].str.count("'")

In [366]:
number_of_title_words=[]
for sentence in df['text']:
    doc=nlp(sentence)
    p=0
    for tokens in doc:
        if tokens.text.istitle() :
            p=p+1
    number_of_title_words.append(p)  

In [367]:
df['Title_Case_Words']=number_of_title_words

In [368]:
df_meta=df[['n_words','Period','Comma','Question_Marks','Colon','Semicolon','Quotation_Mark','Aphostrophe','Title_Case_Words']]

In [369]:
df_stack=pd.concat([df_meta,df_svd_features,df_stacked],axis=1)

In [370]:
df_stack.shape

(19579, 45)

In [373]:
df['Target'].shape

(19579,)

In [380]:
X_training_stack,X_test_stack,Y_training_stack,Y_test_stack=train_test_split(df_stack.get_values(),df['Target'].get_values(),test_size=0.25,random_state=42,shuffle=True)

In [381]:
X_training_stack.shape

(14684, 45)

In [382]:
X_test_stack.shape

(4895, 45)

In [383]:
Y_training_stack.shape

(14684,)

In [384]:
Y_test_stack.shape

(4895,)

** Features for submission data **

In [385]:
df_sub.head(5)

,id,text
0,id02310,"Still, as I urged our leaving Ireland with such inquietude and impatience, my father thought it best to yield."
1,id24541,"If a fire wanted fanning, it could readily be fanned with a newspaper, and as the government grew weaker, I have no doubt that leather and iron acquired durability in proportion, for, in a very short time, there was not a pair of bellows in all Rotterdam that ever stood in need of a stitch or required the assistance of a hammer."
2,id00134,"And when they had broken down the frail door they found only this: two cleanly picked human skeletons on the earthen floor, and a number of singular beetles crawling in the shadowy corners."
3,id27757,"While I was thinking how I should possibly manage without them, one actually tumbled out of my head, and, rolling down the steep side of the steeple, lodged in the rain gutter which ran along the eaves of the main building."
4,id04081,I am not sure to what limit his knowledge may extend.


** TfIdf features for submission data **

In [407]:
Vectorizer=TfidfVectorizer(stop_words='english')
X_train_stack=Vectorizer.fit_transform(df['text'].get_values())
Y_train_stack=df['Target'].get_values()
X_sub_stack=Vectorizer.transform(df_sub['text'].get_values())
mnb_train=MultinomialNB(alpha=0.01)
mnb_train.fit(X_train_stack,Y_train_stack)
predict_proba=mnb_train.predict_proba(X_sub_stack)
df_tfidf_sub_stack=pd.DataFrame(predict_proba[:,[0,1]],columns=['tfidf_mnb_feat1','tfidf_mnb_feat2'])
df_tfidf_sub_stack['tfidf_mnb_feat1']=df_tfidf_sub_stack['tfidf_mnb_feat1'].apply(lambda x :np.round(x,2))
df_tfidf_sub_stack['tfidf_mnb_feat2']=df_tfidf_sub_stack['tfidf_mnb_feat2'].apply(lambda x :np.round(x,2))

**cvt features for submission data**

In [411]:
Cvt_Vectorizer=CountVectorizer(stop_words='english')
Xcvt_train_stack=Cvt_Vectorizer.fit_transform(df['text'].get_values())
Ycvt_train_stack=df['Target'].get_values()
Xcvt_sub_stack=Cvt_Vectorizer.transform(df_sub['text'].get_values())
mnbcvt_train=MultinomialNB(alpha=1)
mnbcvt_train.fit(Xcvt_train_stack,Ycvt_train_stack)
cvt_predict_proba=mnbcvt_train.predict_proba(Xcvt_sub_stack)
df_cvt_sub_stack=pd.DataFrame(cvt_predict_proba[:,[0,1]],columns=['cvt_mnb_feat1','cvt_mnb_feat2'])
df_cvt_sub_stack['cvt_mnb_feat1']=df_cvt_sub_stack['cvt_mnb_feat1'].apply(lambda x :np.round(x,2))
df_cvt_sub_stack['cvt_mnb_feat2']=df_cvt_sub_stack['cvt_mnb_feat2'].apply(lambda x :np.round(x,2))

**Markov features for submission **

In [418]:
dataframe_sub=Conditional_n_grams(list(df['text'].get_values()),list(df['author'].get_values()),4)
predict_proba_mk_sub=Target_Prediction(df_sub['text'].get_values(),4,prior=dataframe_sub,alpha=0.1)

1000
2000
3000
4000
5000
6000
7000
8000


In [420]:
df_sub_mk=pd.DataFrame(predict_proba_mk_sub[:,[0,1]],columns=['mk_sub_feat1','mk_sub_feat2'])
df_sub_mk['mk_sub_feat1']=df_sub_mk['mk_sub_feat1'].apply(lambda x : np.round(x,2))
df_sub_mk['mk_sub_feat2']=df_sub_mk['mk_sub_feat2'].apply(lambda x : np.round(x,2))

In [421]:
df_sub_stacked=pd.concat([df_tfidf_sub_stack,df_cvt_sub_stack,df_sub_mk],axis=1)

In [422]:
df_sub_stacked.head(10)

,tfidf_mnb_feat1,tfidf_mnb_feat2,cvt_mnb_feat1,cvt_mnb_feat2,mk_sub_feat1,mk_sub_feat2
0,0.04,0.01,0.00,0.00,0.00,0.00
1,0.97,0.01,1.00,0.00,1.00,0.00
2,0.60,0.40,0.18,0.82,0.00,1.00
3,0.66,0.34,0.45,0.55,0.09,0.91
4,0.86,0.10,0.97,0.02,0.97,0.00
5,0.80,0.16,0.86,0.14,1.00,0.00
6,0.48,0.36,0.50,0.46,0.99,0.01
7,0.08,0.17,0.00,0.04,0.00,0.00
8,0.99,0.01,1.00,0.00,1.00,0.00
9,0.67,0.19,0.80,0.07,0.99,0.01


** Meta features **

In [416]:
df_sub['n_words']=df_sub['text'].apply(lambda x : len(x.split(" ")))
df_sub['Period']=df_sub['text'].str.count('\.')
df_sub['Comma']=df_sub['text'].str.count(',')
df_sub['Question_Marks']=df_sub['text'].str.count(r'\?')
df_sub['Colon']=df_sub['text'].str.count(r':')
df_sub['Semicolon']=df_sub['text'].str.count(';')
df_sub['Quotation_Mark']=df_sub['text'].str.count('"')
df_sub['Aphostrophe']=df_sub['text'].str.count("'")
number_of_title_words=[]
for sentence in df_sub['text']:
    doc=nlp(sentence)
    p=0
    for tokens in doc:
        if tokens.text.istitle() :
            p=p+1
    number_of_title_words.append(p)  
df_sub['Title_Case_Words']=number_of_title_words

In [417]:
df_sub_meta=df_sub[['n_words','Period','Comma','Question_Marks','Colon','Semicolon','Quotation_Mark','Aphostrophe','Title_Case_Words']]

**LSI features **

In [424]:
X_sub_Tfidf=Vectorizer_stack.transform(df_sub['text'].get_values())
X_sub_svd=svd.transform(X_sub_Tfidf)
svd_sub_columns=[]
for p in np.arange(30):
    svd_sub_columns.append('svd_sub_columns_'+str(p))
df_sub_svd=pd.DataFrame(X_sub_svd,columns=svd_sub_columns)

In [425]:
df_sub_stacked=pd.concat([df_sub_meta,df_sub_svd,df_sub_stacked],axis=1)

In [426]:
df_sub_stacked.shape

(8392, 45)

In [428]:
df_stack.head(10)

,n_words,Period,Comma,Question_Marks,Colon,Semicolon,Quotation_Mark,Aphostrophe,Title_Case_Words,svd_columns_0,...,svd_columns_26,svd_columns_27,svd_columns_28,svd_columns_29,Tfidf_feat1,Tfidf_feat2,cvtNb_feat1,cvtNb_feat2,mk_feat1,mk_feat2
0,41,1,4,0,0,2,0,0,3,0.061037,...,-0.007574,-0.021656,-0.000622,-0.053015,0.99,0.00,1.00,0.00,1.00,0.00
1,14,1,0,0,0,0,0,0,1,0.019324,...,0.000005,-0.007604,0.012291,-0.014414,0.20,0.76,0.88,0.09,0.00,1.00
2,36,1,4,0,0,0,0,0,1,0.066671,...,-0.080321,-0.009206,-0.043403,0.060264,0.96,0.04,0.98,0.02,1.00,0.00
3,34,1,3,0,0,0,0,0,4,0.068588,...,-0.043884,0.025721,-0.050824,-0.034901,0.00,0.00,0.00,0.00,0.00,0.00
4,27,1,2,0,0,1,0,0,2,0.028288,...,-0.000827,0.000556,0.006208,0.002744,0.61,0.36,0.63,0.37,0.53,0.46
5,83,1,4,0,1,0,0,0,5,0.083893,...,-0.095226,0.047396,-0.033463,-0.023350,0.00,0.00,0.00,0.00,0.00,0.00
6,21,1,3,0,0,1,0,0,1,0.050682,...,0.028070,-0.011223,-0.014391,-0.046288,0.79,0.14,0.99,0.01,1.00,0.00
7,8,1,0,0,0,0,0,0,1,0.019461,...,-0.014893,-0.003963,0.002818,-0.014358,0.98,0.01,0.93,0.04,0.81,0.10
8,88,1,7,0,0,1,0,2,7,0.118808,...,0.007728,0.046941,-0.000738,0.041659,0.98,0.02,1.00,0.00,1.00,0.00
9,23,1,2,0,0,0,0,0,1,0.013881,...,0.000365,0.002515,-0.002347,-0.005261,0.29,0.03,0.16,0.02,0.00,0.00


In [430]:
df_sub_stacked.head(10)

,n_words,Period,Comma,Question_Marks,Colon,Semicolon,Quotation_Mark,Aphostrophe,Title_Case_Words,svd_sub_columns_0,...,svd_sub_columns_26,svd_sub_columns_27,svd_sub_columns_28,svd_sub_columns_29,tfidf_mnb_feat1,tfidf_mnb_feat2,cvt_mnb_feat1,cvt_mnb_feat2,mk_sub_feat1,mk_sub_feat2
0,19,1,2,0,0,0,0,0,3,0.055677,...,0.031527,-0.025241,-0.062803,0.033175,0.04,0.01,0.00,0.00,0.00,0.00
1,62,1,6,0,0,0,0,0,3,0.054519,...,-0.002258,0.007305,0.008341,-0.010056,0.97,0.01,1.00,0.00,1.00,0.00
2,33,1,1,0,1,0,0,0,1,0.039503,...,-0.018751,0.022615,0.003829,-0.001938,0.60,0.40,0.18,0.82,0.00,1.00
3,41,1,4,0,0,0,0,0,3,0.029997,...,0.005526,-0.005152,0.000271,-0.010117,0.66,0.34,0.45,0.55,0.09,0.91
4,11,1,0,0,0,0,0,0,1,0.023256,...,-0.007936,0.000834,-0.002123,-0.009083,0.86,0.10,0.97,0.02,0.97,0.00
5,32,1,6,0,0,0,1,0,3,0.034411,...,-0.010588,-0.000419,0.015033,-0.008716,0.80,0.16,0.86,0.14,1.00,0.00
6,13,1,1,0,0,0,0,0,1,0.056038,...,-0.036670,-0.040631,0.143580,0.017636,0.48,0.36,0.50,0.46,0.99,0.01
7,30,1,1,0,0,1,0,0,5,0.143744,...,-0.056955,0.032518,0.018353,-0.012188,0.08,0.17,0.00,0.04,0.00,0.00
8,28,1,4,0,0,0,0,0,1,0.084013,...,0.078137,0.032659,0.076846,0.014953,0.99,0.01,1.00,0.00,1.00,0.00
9,8,1,0,0,0,0,1,0,1,0.033882,...,0.027890,-0.039498,0.039765,-0.094125,0.67,0.19,0.80,0.07,0.99,0.01


** Traning XGboost on stacked data **

In [457]:
XGB=XGBClassifier(max_depth=3,learning_rate=0.1,n_estimators=500,silent=False,objective='multi:softmax',sub_sample=0.8,colsample_bytree=0.8)

In [458]:
XGB.fit(X_training_stack,Y_training_stack,eval_set=[(X_training_stack,Y_training_stack),(X_test_stack,Y_test_stack)],eval_metric='mlogloss',early_stopping_rounds=50)

[0]	validation_0-mlogloss:1.00537	validation_1-mlogloss:1.00712
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-mlogloss:0.924996	validation_1-mlogloss:0.928346
[2]	validation_0-mlogloss:0.856462	validation_1-mlogloss:0.8616
[3]	validation_0-mlogloss:0.798981	validation_1-mlogloss:0.805521
[4]	validation_0-mlogloss:0.748317	validation_1-mlogloss:0.756266
[5]	validation_0-mlogloss:0.704288	validation_1-mlogloss:0.713512
[6]	validation_0-mlogloss:0.66591	validation_1-mlogloss:0.676311
[7]	validation_0-mlogloss:0.632479	validation_1-mlogloss:0.644148
[8]	validation_0-mlogloss:0.604222	validation_1-mlogloss:0.616872
[9]	validation_0-mlogloss:0.578154	validation_1-mlogloss:0.591766
[10]	validation_0-mlogloss:0.55533	validation_1-mlogloss:0.569854
[11]	validation_0-mlogloss:0.534787	validation_1-mlogloss:0.550178
[12]	validation_0-mlogloss:0.516758	validatio

[121]	validation_0-mlogloss:0.300394	validation_1-mlogloss:0.363056
[122]	validation_0-mlogloss:0.299922	validation_1-mlogloss:0.362995
[123]	validation_0-mlogloss:0.299486	validation_1-mlogloss:0.362836
[124]	validation_0-mlogloss:0.299098	validation_1-mlogloss:0.362785
[125]	validation_0-mlogloss:0.298656	validation_1-mlogloss:0.362791
[126]	validation_0-mlogloss:0.298252	validation_1-mlogloss:0.362638
[127]	validation_0-mlogloss:0.297856	validation_1-mlogloss:0.362531
[128]	validation_0-mlogloss:0.297442	validation_1-mlogloss:0.362409
[129]	validation_0-mlogloss:0.2971	validation_1-mlogloss:0.362356
[130]	validation_0-mlogloss:0.296804	validation_1-mlogloss:0.362339
[131]	validation_0-mlogloss:0.296481	validation_1-mlogloss:0.362332
[132]	validation_0-mlogloss:0.296008	validation_1-mlogloss:0.362376
[133]	validation_0-mlogloss:0.295689	validation_1-mlogloss:0.362326
[134]	validation_0-mlogloss:0.295152	validation_1-mlogloss:0.362241
[135]	validation_0-mlogloss:0.29474	validation_1-m

[242]	validation_0-mlogloss:0.261845	validation_1-mlogloss:0.360347
[243]	validation_0-mlogloss:0.261679	validation_1-mlogloss:0.360347
[244]	validation_0-mlogloss:0.261543	validation_1-mlogloss:0.360307
[245]	validation_0-mlogloss:0.261152	validation_1-mlogloss:0.360268
[246]	validation_0-mlogloss:0.260805	validation_1-mlogloss:0.360307
[247]	validation_0-mlogloss:0.260573	validation_1-mlogloss:0.3604
[248]	validation_0-mlogloss:0.260254	validation_1-mlogloss:0.360344
[249]	validation_0-mlogloss:0.260018	validation_1-mlogloss:0.360445
[250]	validation_0-mlogloss:0.259777	validation_1-mlogloss:0.360403
[251]	validation_0-mlogloss:0.259422	validation_1-mlogloss:0.360376
[252]	validation_0-mlogloss:0.259124	validation_1-mlogloss:0.360381
[253]	validation_0-mlogloss:0.258831	validation_1-mlogloss:0.360429
[254]	validation_0-mlogloss:0.258574	validation_1-mlogloss:0.360417
[255]	validation_0-mlogloss:0.258366	validation_1-mlogloss:0.360408
[256]	validation_0-mlogloss:0.258072	validation_1-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, sub_sample=0.8, subsample=1)

with learning rate on 0.1, we can see that the 245 trees is giving the best result, using this we will tune for max_depth and sampling

In [459]:
XGB1=XGBClassifier(max_depth=3,learning_rate=0.1,n_estimators=245,silent=False,objective='multi:softmax',sub_sample=0.8,colsample_bytree=0.8)

In [460]:
parameter_grid={'sub_sample':[0.5,0.6],'colsample_bytree':[0.5,0.6]}

In [461]:
clf=GridSearchCV(XGB1,param_grid=parameter_grid,scoring='log_loss',cv=5)

In [462]:
clf.fit(X_training_stack,Y_training_stack)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=245,
       n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, sub_sample=0.8, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'colsample_bytree': [0.5, 0.6], 'sub_sample': [0.5, 0.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='log_loss', verbose=0)

In [463]:
clf.cv_results_

{'mean_fit_time': array([ 8.93824382, 31.25310559, 10.3675189 , 10.42822018]),
 'mean_score_time': array([0.08146119, 0.08166819, 0.08077669, 0.08022943]),
 'mean_test_score': array([-0.34050062, -0.34050062, -0.34181932, -0.34181932]),
 'mean_train_score': array([-0.25376899, -0.25376899, -0.2525625 , -0.2525625 ]),
 'param_colsample_bytree': masked_array(data=[0.5, 0.5, 0.6, 0.6],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_sub_sample': masked_array(data=[0.5, 0.6, 0.5, 0.6],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'colsample_bytree': 0.5, 'sub_sample': 0.5},
  {'colsample_bytree': 0.5, 'sub_sample': 0.6},
  {'colsample_bytree': 0.6, 'sub_sample': 0.5},
  {'colsample_bytree': 0.6, 'sub_sample': 0.6}],
 'rank_test_score': array([1, 1, 3, 3], dtype=int32),
 'split0_test_score': array([-0.33030001, -0.33030001, -0.33240457, -0.33240457]),
 'split0_tr

In [464]:
print("best cv score: %.2f best parameters: %s "%(clf.best_score_,clf.best_params_))

best cv score: -0.34 best parameters: {'colsample_bytree': 0.5, 'sub_sample': 0.5} 


In [465]:
log_loss(Y_test_stack,clf.predict_proba(X_test_stack))

0.36041591598071

In [473]:
accuracy_score(Y_test_stack,clf.predict(X_test_stack))

0.8602655771195097

using stacked models has improved logloss score from 0.48 to 0.36, but the over all accuracy improvement is marginal only 4%

** Summary **

For author identification, we have taken multi pronged approach to tackle the problem and finally stacked all the models together , which has improved accuracy and logloss score.

Tfidf vectorizer with Naive bayes algorithm : Accuracy 81.2% logloss score:0.460

Cvt vectorizer with Naive bayes algorithm : Accuracy 83.9% logloss score: 0.480

Markov chain with order 3 : Accuracy Accuracy 78.9% logloss score:1.05
We have used LSI to reduce the dimensionality and this can be used build a independant model, Instead we have used it to stack 
with other models.

Finally we have stack LSI,Tfidf,Cvt,LSI features and applied XGboost , which has resulted in highest accuracy:86% and highest logloss score:0.36

For practical purposes, cvt model will be sufficient because it's accuracy is only 2% less and the model is simple,unlike the stacked model.

**Next steps**

The model accuracy can be further improved by trying the following options:
    
    1) Tune min_df and max_df parameters in Count vectorizer and TfIdf vectorizer
    
    2) Tune the number of dimensions in LSI using an independant model
    
    3) In the markov model we assumed order 3 and alpha of 0.1. These parameters can be tuned further for improved accuracy
    
Over all the classifier built above serves practical purposes , but can be further improved.